In [1]:
from tqdm import tqdm
from advsecurenet.models.model_factory import ModelFactory
from advsecurenet.datasets import DatasetFactory
from advsecurenet.dataloader import DataLoaderFactory
from advsecurenet.shared.types import DatasetType
from advsecurenet.utils.model_utils import save_model
from advsecurenet.shared.types.configs.train_config import TrainConfig
from advsecurenet.defenses import AdversarialTraining
from advsecurenet.attacks.fgsm import FGSM
from advsecurenet.attacks.pgd import PGD
from advsecurenet.attacks.lots import LOTS
from advsecurenet.shared.types.configs.defense_configs.adversarial_training_config import AdversarialTrainingConfig
import advsecurenet.shared.types.configs.attack_configs as AttackConfigs
from advsecurenet.utils.tester import Tester
from advsecurenet.utils.trainer import Trainer



/home/user/catal/miniconda3/envs/new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load basic MNIST model
mnist_model = ModelFactory.create_model(model_name='CustomMnistModel', num_classes=10, num_input_channels=1)
mnist_model

CustomModel(
  (model): CustomMnistModel(
    (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fc1): Linear(in_features=50176, out_features=512, bias=True)
    (relu): ReLU()
    (fc2): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [3]:
# load MNIST dataset
dataset = DatasetFactory.create_dataset(DatasetType.MNIST)
train_data = dataset.load_dataset(train=True)
test_data = dataset.load_dataset(train=False)
train_loader = DataLoaderFactory.create_dataloader(dataset=train_data, batch_size=128, shuffle=True)
test_loader = DataLoaderFactory.create_dataloader(dataset=test_data, batch_size=128, shuffle=False)
print(f"Train dataset size: {len(train_data)}")
print(f"Test dataset size: {len(test_data)}")

Train dataset size: 60000
Test dataset size: 10000


In [4]:
# first normal training
train_config = TrainConfig(model= mnist_model, train_loader=train_loader, epochs=1, device="cuda:2")
trainer = Trainer(train_config)
trainer.train()

100%|██████████| 469/469 [00:10<00:00, 44.13it/s]

Epoch 1 loss: 0.14634578182562583


In [5]:
# Function to test model robustness against different attacks
def test_model_robustness(model, test_loader, attack, device):
    model.eval()  # Set the model to evaluation mode

    # Initialize counters
    correct = 0
    adv_correct = 0
    total = 0

    for data, target in tqdm(test_loader, desc='Testing'):
        # Send data and target to the same device as your model
        data, target = data.to(device), target.to(device)
        
        # Get the original model's predictions
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        
        # Generate adversarial data using the provided attack method
        fgsm_data = attack.attack(model=model, x=data, y=target)
        
        # Get the model's predictions on the adversarial data
        adv_output = model(fgsm_data)
        adv_pred = adv_output.argmax(dim=1, keepdim=True)
        adv_correct += adv_pred.eq(target.view_as(adv_pred)).sum().item()

        total += target.size(0)

    # Calculate the original accuracy
    original_accuracy = correct / total

    # Calculate the adversarial accuracy
    adversarial_accuracy = adv_correct / total

    # Calculate the robustness as the difference in accuracies
    robustness = original_accuracy - adversarial_accuracy

    print(f'Original Accuracy: {original_accuracy:.2%}')
    print(f'Adversarial Accuracy: {adversarial_accuracy:.2%}')
    print(f'Robustness (Accuracy Drop): {robustness:.2%}')


In [6]:
# get FGSM attack
fgsm_config = AttackConfigs.FgsmAttackConfig(epsilon=0.5, device="cuda:2")
fgsm = FGSM(fgsm_config)

In [7]:
# Testing base model against FGSM attack 
test_model_robustness(mnist_model, test_loader, fgsm, device="cuda:2")

Testing: 100%|██████████| 79/79 [00:01<00:00, 47.02it/s]

Original Accuracy: 98.45%
Adversarial Accuracy: 88.34%
Robustness (Accuracy Drop): 10.11%


In [8]:
# use fgsm to adversarially train the model
robust_model = ModelFactory.create_model(model_name='CustomMnistModel', num_classes=10, num_input_channels=1)
adversarial_training_config = AdversarialTrainingConfig(model=robust_model, models=[robust_model], attacks=[fgsm], train_loader=train_loader, epochs=1, device="cuda:2")
adversarial_training = AdversarialTraining(adversarial_training_config)
adversarial_training.train()

Running epoch 1...


Training: 100%|██████████| 469/469 [00:16<00:00, 28.10it/s]

Epoch 1/1 Loss: 0.14964635183077568


In [9]:
# testing the clean accuracy of the adversarially trained model
tester = Tester(model=robust_model, test_loader=test_loader, device="cuda:2")
tester.test()

Testing on cuda:2


Testing: 100%|██████████| 79/79 [00:00<00:00, 79.32batch/s] 


Test set: Average loss: 0.0004, Accuracy: 9809/10000 (98.09%)


(0.000422191862994805, 98.09)

In [10]:
# generate the adversarial image using PGD attack
target_layer = "model.fc2" # this is the name of the layer that we want to target - this assumes that the model has a layer named fc2
lots_config = AttackConfigs.LotsAttackConfig(
    deep_feature_layer=target_layer,
    mode = AttackConfigs.LotsAttackMode.SINGLE,
    max_iterations=10000000,
    learning_rate=0.1,
    epsilon=0.01,
    device = "cuda:2"
)
lots = LOTS(lots_config)

In [11]:
# use fgsm to adversarially train the model
robust_model = ModelFactory.create_model(model_name='CustomMnistModel', num_classes=10, num_input_channels=1)
adversarial_training_config = AdversarialTrainingConfig(model=robust_model, models=[robust_model], attacks=[lots], train_loader=train_loader, epochs=1, device="cuda:2")
adversarial_training = AdversarialTraining(adversarial_training_config)
adversarial_training.train()

Running epoch 1...


Training: 100%|██████████| 469/469 [00:29<00:00, 15.72it/s]


Epoch 1/1 Loss: 1.2340536783498997


In [12]:
# testing the clean accuracy of the adversarially trained model
tester = Tester(model=robust_model, test_loader=test_loader, device="cuda:2")
tester.test()

Testing on cuda:2


Testing: 100%|██████████| 79/79 [00:00<00:00, 82.39batch/s] 


Test set: Average loss: 0.0004, Accuracy: 9823/10000 (98.23%)


(0.00042206173918093553, 98.23)

In [13]:
# Testing base model against FGSM attack 
test_model_robustness(robust_model, test_loader, fgsm, device="cuda:2")

Testing: 100%|██████████| 79/79 [00:01<00:00, 46.51it/s]

Original Accuracy: 98.23%
Adversarial Accuracy: 11.47%
Robustness (Accuracy Drop): 86.76%
